In [1]:
# For GS1, the retrieval step takes about 30 minutes to run and requires 1.9 GB of space.

In [2]:
import grizli
from hsaquery import query, overlaps
import os
import numpy as np
from grizli.pipeline import auto_script
import glob
from glob import glob
import astropy
from grizli.prep import process_direct_grism_visit
from astropy.io import fits

In [3]:
field           = 'GS1'
ref_filter      = 'F105W'


###
# NEED TO CHANGE THESE TO YOUR OWN WORKING DIRECTORIES.
# PATH_TO_CATS should include the 
#     reference mosaic image (e.g., goodss-F105W-astrodrizzle-v4.3_drz_sci.fits)
#     segmentation map       (e.g., Goods_S_plus_seg.fits)
#     source catalog         (e.g., goodss-F105W-astrodrizzle-v4.3_drz_sub_plus.cat)
#     radec_catalog          (e.g., goodsS_radec.cat)
#     3DHST Eazy Catalogs    (e.g., goodss_3dhst.v4.1.cats/*)
# these are all available on the team archive: https://archive.stsci.edu/pub/clear_team/CATALOGS/
# 
##

HOME_PATH       = '/Users/rsimons/Desktop/clear/for_hackday/%s'%field
PATH_TO_CATS    = '/Users/rsimons/Desktop/clear/Catalogs'

if not os.path.isdir(HOME_PATH): os.system('mkdir %s'%HOME_PATH)
if not os.path.isdir(HOME_PATH + '/query_results'): os.system('mkdir %s/query_results'%HOME_PATH)

os.chdir(HOME_PATH)


In [4]:
if False:
    extra = query.DEFAULT_EXTRA.copy()
    extra += ["TARGET.TARGET_NAME LIKE '%s'"%field]

    parent = query.run_query(proposid = [14227], instruments=['WFC3-IR', 'ACS-WFC'], 
                         extensions=['FLT'], filters=['G102'], extra = extra)


    tabs = overlaps.find_overlaps(parent, buffer_arcmin=0.01, filters=['G102', 'G141'], 
                                  instruments=['WFC3-IR','WFC3-UVIS','ACS-WFC'], extra=[], 
                                  close=False)

    footprint_fits_file = glob('*footprint.fits')[0]
    jtargname = footprint_fits_file.strip('_footprint.fits')

    fp_fits = fits.open(footprint_fits_file)

    overlapping_target_names = set(fp_fits[1].data['target'])
    s3_status = os.system('aws s3 ls s3://stpubdata --request-payer requester')
    os.system('cp %s/%s_footprint.fits %s/query_results/%s_footprint_%s.fits'%(HOME_PATH, jtargname, HOME_PATH, jtargname, 'all_G102_G141'))
    os.system('cp %s/%s_footprint.npy %s/query_results/%s_footprint_%s.npy'%(HOME_PATH, jtargname, HOME_PATH, jtargname,  'all_G102_G141'))
    os.system('cp %s/%s_footprint.pdf %s/query_results/%s_footprint_%s.pdf'%(HOME_PATH, jtargname, HOME_PATH, jtargname,  'all_G102_G141'))
    os.system('cp %s/%s_info.dat %s/query_results/%s_info_%s.dat'%(HOME_PATH, jtargname, HOME_PATH, jtargname,  'all_G102_G141'))


In [5]:
if False:
    for t, targ_name in enumerate(overlapping_target_names):
        extra = query.DEFAULT_EXTRA.copy()
        extra += ["TARGET.TARGET_NAME LIKE '%s'"%field]
        tabs = overlaps.find_overlaps(parent, buffer_arcmin=0.01, filters=['G102', 'G141', 'F098M', 'F105W', 'F125W', 'F140W'], 
                                      instruments=['WFC3-IR','WFC3-UVIS','ACS-WFC'], extra=extra, close=False)
        s3_status = os.system('aws s3 ls s3://stpubdata --request-payer requester')
        auto_script.fetch_files(field_root=jtargname, HOME_PATH=HOME_PATH, remove_bad=True, 
                                reprocess_parallel=True, s3_sync=(s3_status == 0))
        os.system('mv %s/%s_footprint.fits %s/query_results/%s_footprint_%s.fits'%(HOME_PATH, jtargname, HOME_PATH, jtargname, targ_name))
        os.system('mv %s/%s_footprint.npy %s/query_results/%s_footprint_%s.npy'%(HOME_PATH, jtargname, HOME_PATH, jtargname, targ_name))
        os.system('mv %s/%s_footprint.pdf %s/query_results/%s_footprint_%s.pdf'%(HOME_PATH, jtargname, HOME_PATH, jtargname, targ_name))
        os.system('mv %s/%s_info.dat %s/query_results/%s_info_%s.dat'%(HOME_PATH, jtargname, HOME_PATH, jtargname, targ_name))

        os.chdir(HOME_PATH)

In [6]:
# PATH_TO_RAW and PATH_TO_PREP are created in the previous steps

PATH_TO_RAW     = glob(HOME_PATH + '/*/RAW')[0]
PATH_TO_PREP    = glob(HOME_PATH + '/*/PREP')[0]


class Pointing():
    """ Generalization of GN1, GS1, ERSPRIME, etc

    To change field-dependent catalog, seg map, ref image, and padding
    only need to change them here.

    """
    def __init__(self, field, ref_filter):
        if 'N' in field.upper():
            self.pad = 500
            self.radec_catalog = PATH_TO_CATS + '/old_radeccats/goodsN_radec.cat'
            self.seg_map =  PATH_TO_CATS + '/Goods_N_plus_seg.fits'
            self.catalog =  PATH_TO_CATS + '/goodsn-F105W-astrodrizzle-v4.4_drz_sub_plus.cat'
            self.ref_image =  PATH_TO_CATS + '/goodsn-F105W-astrodrizzle-v4.4_drz_sci.fits'
            
            self.tempfilt, self.coeffs, self.temp_sed, self.pz = readEazyBinary(MAIN_OUTPUT_FILE='goodsn_3dhst.v4.1', OUTPUT_DIRECTORY=PATH_TO_CATS, CACHE_FILE='Same')
            self.params = {}
            self.params['CATALOG_FILE'] = PATH_TO_CATS + '/{0}_3dhst.{1}.cats/Catalog/{0}_3dhst.{1}.cat'.format('goodsn', 'v4.1')
            self.params['Z_STEP'] = 0.002
            self.params['Z_MAX'] = 4
            self.params['MAIN_OUTPUT_FILE'] = '{0}_3dhst.{1}.eazypy'.format('goodsn', 'v4.1')
            self.params['PRIOR_FILTER'] = 205
            self.params['MW_EBV'] = {'aegis':0.0066, 'cosmos':0.0148, 'goodss':0.0069, 
                                    'uds':0.0195, 'goodsn':0.0103}['goodsn']
            self.params['TEMPLATES_FILE'] = 'templates/fsps_full/tweak_fsps_QSF_12_v3.param'
            self.translate_file = PATH_TO_CATS + '/{0}_3dhst.{1}.cats/Eazy/{0}_3dhst.{1}.translate'.format('goodsn', 'v4.1')

        elif 'S' in field.upper():
            self.pad = 200 # grizli default
            self.radec_catalog =  PATH_TO_CATS + '/old_radeccats/goodsS_radec.cat'
            self.seg_map =  PATH_TO_CATS + '/Goods_S_plus_seg.fits'
            self.catalog =  PATH_TO_CATS + '/goodss-F105W-astrodrizzle-v4.3_drz_sub_plus.cat'
            self.ref_image =  PATH_TO_CATS + '/goodss-F105W-astrodrizzle-v4.3_drz_sci.fits'


            self.tempfilt, self.coeffs, self.temp_sed, self.pz = readEazyBinary(MAIN_OUTPUT_FILE='goodss_3dhst.v4.1', OUTPUT_DIRECTORY=PATH_TO_CATS, CACHE_FILE='Same')
            self.params = {}
            self.params['CATALOG_FILE'] = PATH_TO_CATS + '/{0}_3dhst.{1}.cats/Catalog/{0}_3dhst.{1}.cat'.format('goodss', 'v4.1')
            self.params['Z_STEP'] = 0.002
            self.params['Z_MAX'] = 4
            self.params['MAIN_OUTPUT_FILE'] = '{0}_3dhst.{1}.eazypy'.format('goodss', 'v4.1')
            self.params['PRIOR_FILTER'] = 205
            self.params['MW_EBV'] = {'aegis':0.0066, 'cosmos':0.0148, 'goodss':0.0069, 
                                    'uds':0.0195, 'goodsn':0.0103}['goodss']
            self.params['TEMPLATES_FILE'] = 'templates/fsps_full/tweak_fsps_QSF_12_v3.param'
            self.translate_file = PATH_TO_CATS + '/{0}_3dhst.{1}.cats/Eazy/{0}_3dhst.{1}.translate'.format('goodss', 'v4.1')




def readEazyBinary(MAIN_OUTPUT_FILE='photz', OUTPUT_DIRECTORY='./OUTPUT', CACHE_FILE='Same'):

    """
    Author: Gabe Brammer
    This function has been clipped from eazyPy.py in thethreedhst git respository
    https://github.com/gbrammer/threedhst/tree/master/threedhst

    tempfilt, coeffs, temp_sed, pz = readEazyBinary(MAIN_OUTPUT_FILE='photz', \
                                                OUTPUT_DIRECTORY='./OUTPUT', \
                                                CACHE_FILE = 'Same')

    Read Eazy BINARY_OUTPUTS files into structure data.
    
    If the BINARY_OUTPUTS files are not in './OUTPUT', provide either a relative or absolute path
    in the OUTPUT_DIRECTORY keyword.
    
    By default assumes that CACHE_FILE is MAIN_OUTPUT_FILE+'.tempfilt'.
    Specify the full filename if otherwise. 
    """
    
    #root='COSMOS/OUTPUT/cat3.4_default_lines_zp33sspNoU'
    
    root = OUTPUT_DIRECTORY+'/'+MAIN_OUTPUT_FILE
    
    ###### .tempfilt
    if CACHE_FILE == 'Same':
        CACHE_FILE = root+'.tempfilt'
    
    if os.path.exists(CACHE_FILE) is False:
        print(('File, %s, not found.' %(CACHE_FILE)))
        return -1,-1,-1,-1
    
    f = open(CACHE_FILE,'rb')
    
    s = np.fromfile(file=f,dtype=np.int32, count=4)
    NFILT=s[0]
    NTEMP=s[1]
    NZ=s[2]
    NOBJ=s[3]
    tempfilt = np.fromfile(file=f,dtype=np.double,count=NFILT*NTEMP*NZ).reshape((NZ,NTEMP,NFILT)).transpose()
    lc = np.fromfile(file=f,dtype=np.double,count=NFILT)
    zgrid = np.fromfile(file=f,dtype=np.double,count=NZ)
    fnu = np.fromfile(file=f,dtype=np.double,count=NFILT*NOBJ).reshape((NOBJ,NFILT)).transpose()
    efnu = np.fromfile(file=f,dtype=np.double,count=NFILT*NOBJ).reshape((NOBJ,NFILT)).transpose()
    
    f.close()
    
    tempfilt  = {'NFILT':NFILT,'NTEMP':NTEMP,'NZ':NZ,'NOBJ':NOBJ,\
                 'tempfilt':tempfilt,'lc':lc,'zgrid':zgrid,'fnu':fnu,'efnu':efnu}
    
    ###### .coeff
    f = open(root+'.coeff','rb')
    
    s = np.fromfile(file=f,dtype=np.int32, count=4)
    NFILT=s[0]
    NTEMP=s[1]
    NZ=s[2]
    NOBJ=s[3]
    coeffs = np.fromfile(file=f,dtype=np.double,count=NTEMP*NOBJ).reshape((NOBJ,NTEMP)).transpose()
    izbest = np.fromfile(file=f,dtype=np.int32,count=NOBJ)
    tnorm = np.fromfile(file=f,dtype=np.double,count=NTEMP)
    
    f.close()
    
    coeffs = {'NFILT':NFILT,'NTEMP':NTEMP,'NZ':NZ,'NOBJ':NOBJ,\
              'coeffs':coeffs,'izbest':izbest,'tnorm':tnorm}
              
    ###### .temp_sed
    f = open(root+'.temp_sed','rb')
    s = np.fromfile(file=f,dtype=np.int32, count=3)
    NTEMP=s[0]
    NTEMPL=s[1]
    NZ=s[2]
    templam = np.fromfile(file=f,dtype=np.double,count=NTEMPL)
    temp_seds = np.fromfile(file=f,dtype=np.double,count=NTEMPL*NTEMP).reshape((NTEMP,NTEMPL)).transpose()
    da = np.fromfile(file=f,dtype=np.double,count=NZ)
    db = np.fromfile(file=f,dtype=np.double,count=NZ)
    
    f.close()
    
    temp_sed = {'NTEMP':NTEMP,'NTEMPL':NTEMPL,'NZ':NZ,\
              'templam':templam,'temp_seds':temp_seds,'da':da,'db':db}
              
    ###### .pz
    if os.path.exists(root+'.pz'):
        f = open(root+'.pz','rb')
        s = np.fromfile(file=f,dtype=np.int32, count=2)
        NZ=s[0]
        NOBJ=s[1]
        chi2fit = np.fromfile(file=f,dtype=np.double,count=NZ*NOBJ).reshape((NOBJ,NZ)).transpose()

        ### This will break if APPLY_PRIOR No
        s = np.fromfile(file=f,dtype=np.int32, count=1)
        
        if len(s) > 0:
            NK = s[0]
            kbins = np.fromfile(file=f,dtype=np.double,count=NK)
            priorzk = np.fromfile(file=f, dtype=np.double, count=NZ*NK).reshape((NK,NZ)).transpose()
            kidx = np.fromfile(file=f,dtype=np.int32,count=NOBJ)
            pz = {'NZ':NZ,'NOBJ':NOBJ,'NK':NK, 'chi2fit':chi2fit, 'kbins':kbins, 'priorzk':priorzk,'kidx':kidx}
        else:
            pz = None
        
        f.close()
        
    else:
        pz = None
    
    if False:
        f = open(root+'.zbin','rb')
        s = np.fromfile(file=f,dtype=np.int32, count=1)
        NOBJ=s[0]
        z_a = np.fromfile(file=f,dtype=np.double,count=NOBJ)
        z_p = np.fromfile(file=f,dtype=np.double,count=NOBJ)
        z_m1 = np.fromfile(file=f,dtype=np.double,count=NOBJ)
        z_m2 = np.fromfile(file=f,dtype=np.double,count=NOBJ)
        z_peak = np.fromfile(file=f,dtype=np.double,count=NOBJ)
        f.close()
        
    ###### Done.    
    return tempfilt, coeffs, temp_sed, pz



In [7]:
os.chdir(PATH_TO_PREP)
files = glob('%s/*flt.fits'%PATH_TO_RAW)
info = grizli.utils.get_flt_info(files)
visits, filters = grizli.utils.parse_flt_files(info=info, uniquename=True)


goodss-01-bhj-01-177.0-F140W 4
goodss-01-bhj-01-177.0-G141 4
GS1-cxt-09-227.0-F105W 2
GS1-cxt-10-227.0-F105W 2
GS1-cxt-07-249.0-F105W 2
GS1-cxt-08-249.0-F105W 2
GS1-cxt-11-269.0-F105W 2
GS1-cxt-12-269.0-F105W 2
GS1-cxt-09-227.0-G102 4
GS1-cxt-10-227.0-G102 4
GS1-cxt-07-249.0-G102 4
GS1-cxt-08-249.0-G102 4
GS1-cxt-11-269.0-G102 4
GS1-cxt-12-269.0-G102 4


In [8]:
if True:
    product_names = np.array([visit['product'] for visit in visits])
    filter_names = np.array([visit['product'].split('-')[-1] for visit in visits])
    basenames = np.array([visit['product'].split('.')[0]+'.0' for visit in visits])




    for ref_grism, ref_filter in [('G102', 'F105W'), ('G141', 'F140W')]:
        print(ref_grism, ref_filter)

        for v, visit in enumerate(visits):
            product = product_names[v]
            basename = basenames[v]
            filt1 = filter_names[v]
            field_in_contest = basename.split('-')[0]
            if (ref_filter.lower() == filt1.lower()):
                #Found a direct image, now search for grism counterpart
                grism_index= np.where((basenames == basename) & (filter_names == ref_grism.lower()))[0][0]
                p = Pointing(field = field, ref_filter = ref_filter)
                radec_catalog = p.radec_catalog
                print (field_in_contest, visits[grism_index])
                status = process_direct_grism_visit(direct = visit,
                                                    grism = visits[grism_index],
                                                    radec = radec_catalog, 
                                                    align_mag_limits = [14, 23])



G102 F105W
File, /Users/rsimons/Desktop/clear/Catalogs/goodss_3dhst.v4.1.tempfilt, not found.
gs1 OrderedDict([('product', 'gs1-cxt-09-227.0-g102'), ('files', ['icxt09f0q_flt.fits', 'icxt09f2q_flt.fits', 'icxt09f5q_flt.fits', 'icxt09f7q_flt.fits'])])
The following task in the stsci.skypac package can be run with TEAL:
                                    skymatch                                    
The following tasks in the drizzlepac package can be run with TEAL:
    astrodrizzle       imagefindpars           mapreg              photeq       
     pixreplace           pixtopix            pixtosky        refimagefindpars  
     resetbits          runastrodriz          skytopix           tweakback      
      tweakreg           updatenpol
../RAW/icxt09ezq_flt.fits -> icxt09ezq_flt.fits  / bpix: $iref/badpix_spars200_Nov9.fits
- IDCTAB: Distortion model from row 1 for chip 1 : F105W
- IDCTAB: Distortion model from row 1 for chip 1 : F105W
../RAW/icxt09f4q_flt.fits -> icxt09f4q_flt.fits  

           Subtract Sky          0.3976 sec.
       Separate Drizzle          1.0339 sec.
          Create Median          0.5082 sec.
                   Blot          1.0345 sec.
                Driz_CR          1.0564 sec.
          Final Drizzle          2.4789 sec.
   ====================          ====================
                  Total          7.2902 sec.
Trailer file written to:  astrodrizzle.log
Image AB zeropoint: 26.269
SEP: err_scale=0.617
   SEP: Extract...


/anaconda2/envs/grizli/lib/python3.5/site-packages/grizli-0.6.0_177_g174db7d-py3.5-macosx-10.6-x86_64.egg/grizli/prep.py:942: RuntimeWarning: invalid value encountered in sqrt
  err = 1/np.sqrt(wht_data)
/anaconda2/envs/grizli/lib/python3.5/site-packages/grizli-0.6.0_177_g174db7d-py3.5-macosx-10.6-x86_64.egg/grizli/prep.py:942: RuntimeWarning: divide by zero encountered in true_divide
  err = 1/np.sqrt(wht_data)
/anaconda2/envs/grizli/lib/python3.5/site-packages/grizli-0.6.0_177_g174db7d-py3.5-macosx-10.6-x86_64.egg/grizli/prep.py:969: RuntimeWarning: divide by zero encountered in true_divide
  ratio = bkg.rms()/err


    Done.


/anaconda2/envs/grizli/lib/python3.5/site-packages/astropy/table/column.py:965: RuntimeWarning: invalid value encountered in less_equal
  return getattr(self.data, op)(other)
/anaconda2/envs/grizli/lib/python3.5/site-packages/grizli-0.6.0_177_g174db7d-py3.5-macosx-10.6-x86_64.egg/grizli/prep.py:1202: TableReplaceWarning: replaced column 'cflux' which looks like an array slice. The new column no longer shares memory with the original array.
  tab[c] *= 1. / uJy_to_dn
/anaconda2/envs/grizli/lib/python3.5/site-packages/grizli-0.6.0_177_g174db7d-py3.5-macosx-10.6-x86_64.egg/grizli/prep.py:1202: TableReplaceWarning: replaced column 'flux' which looks like an array slice. The new column no longer shares memory with the original array.
  tab[c] *= 1. / uJy_to_dn
/anaconda2/envs/grizli/lib/python3.5/site-packages/grizli-0.6.0_177_g174db7d-py3.5-macosx-10.6-x86_64.egg/grizli/prep.py:1202: TableReplaceWarning: replaced column 'peak' which looks like an array slice. The new column no longer share

...



AstroDrizzle Version 2.2.4 is finished processing at 21:54:15.862 (11/10/2018).


   --------------------          --------------------
                   Step          Elapsed time
   --------------------          --------------------

         Initialization          1.2385 sec.
            Static Mask          0.1004 sec.
           Subtract Sky          0.5680 sec.
       Separate Drizzle          1.0442 sec.
          Create Median          0.8370 sec.
                   Blot          2.2864 sec.
                Driz_CR          1.1063 sec.
          Final Drizzle          3.6658 sec.
   ====================          ====================
                  Total          10.8465 sec.
Trailer file written to:  astrodrizzle.log
goodss-01-bhj-01-177.0-g141: EXTVER=1 / ['zodi_G141_clean.fits'] / ['excess_lo_G141_clean.fits', 'G141_scattered_light.fits']


/anaconda2/envs/grizli/lib/python3.5/site-packages/grizli-0.6.0_177_g174db7d-py3.5-macosx-10.6-x86_64.egg/grizli/prep.py:3222: RuntimeWarning: divide by zero encountered in true_divide
  wht[i*Npix:(i+1)*Npix] = 1./(flt['ERR',ext].data**2*dq_mask).flatten()
/anaconda2/envs/grizli/lib/python3.5/site-packages/grizli-0.6.0_177_g174db7d-py3.5-macosx-10.6-x86_64.egg/grizli/prep.py:3260: RuntimeWarning: invalid value encountered in sqrt
  resid = (data-model)*np.sqrt(wht)
/anaconda2/envs/grizli/lib/python3.5/site-packages/grizli-0.6.0_177_g174db7d-py3.5-macosx-10.6-x86_64.egg/grizli/prep.py:3275: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  out = np.linalg.lstsq(A[mask & obj_mask,:], data[mask & obj_mask])


   goodss-01-bhj-01-177.0-g141 > Iter: 1, masked: 30%, [0.85945541 0.01268524 0.         0.02301353 0.         0.00856197
 0.         0.         0.        ]
   goodss-01-bhj-01-177.0-g141 > Iter: 2, masked: 29%, [ 0.8636274   0.00349205 -0.02696155  0.01323213 -0.01117077 -0.00116855
 -0.01915414 -0.00965632 -0.02282706]
   goodss-01-bhj-01-177.0-g141 > Iter: 3, masked: 28%, [ 0.8631482   0.0031868  -0.03014118  0.01283292 -0.01213213 -0.00157303
 -0.02063589 -0.0100323  -0.02589959]
   goodss-01-bhj-01-177.0-g141 > Iter: 4, masked: 28%, [ 0.8629854   0.00323138 -0.03081852  0.01285565 -0.01222759 -0.00154036
 -0.02064229 -0.00999559 -0.02569002]
   goodss-01-bhj-01-177.0-g141 > Iter: 5, masked: 28%, [ 0.86284953  0.00335318 -0.03081082  0.01296646 -0.01212323 -0.0014206
 -0.02069018 -0.00988139 -0.02558754]


/anaconda2/envs/grizli/lib/python3.5/site-packages/numpy/core/fromnumeric.py:688: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)


Setting up logfile :  astrodrizzle.log
AstroDrizzle Version 2.2.4 (2018-06-28 10:36:33 -0400) started at: 21:54:25.486 (11/10/2018)

==== Processing Step  Initialization  started at  21:54:25.491 (11/10/2018)
WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN'  'DEC--TAN'  
CRVAL : 53.25700163000495  -27.68651757938557  
CRPIX : 549.0  484.5  
CD1_1 CD1_2  : 2.647447168162814e-05  -2.3837848703334823e-05  
CD2_1 CD2_2  : -2.3837848703334823e-05  -2.647447168162814e-05  
NAXIS : 1098  969
********************************************************************************
*
*  Estimated memory usage:  up to 80 Mb.
*  Output image size:       1098 X 969 pixels. 
*  Output image file:       ~ 12 Mb. 
*  Cores available:         4
*
********************************************************************************
==== Processing Step  Initialization  finished at  21:54:26.843 (11/10/2018)

==== Processing Step  Static Mask  started at  21:54:26.846 (11/10/2018)
==== Processing Step  Static M